# Part 1

### Setup to read from Wikipedia

In [2]:
pip install wikipedia

     |████████████████████████████████| 112kB 27.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

     |████████████████████████████████| 5.8MB 8.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import wikipedia
import pandas as pd

### Read table into a dataframe

In [42]:
page = wikipedia.page("List of postal codes of Canada: M").html().encode("UTF-8")
toronto_df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df = pd.read_html(page)[0]
toronto_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


### Remove all rows where Borough = Not assigned

In [43]:
toronto_df.drop(toronto_df[toronto_df["Borough"]=="Not assigned"].index, inplace=True)
toronto_df.reset_index(drop=True, inplace=True)
toronto_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


### Assigning Borough value to Neighborhood where Neighborhood = Not assigned

In [44]:
for row in toronto_df.index:
    if toronto_df.iloc[row, 2] == "Not assigned":
        toronto_df.iloc[row, 2] = toronto_df.iloc[row, 1]
toronto_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


### Combining Neighborhood column for rows of same Postal Code

In [45]:
toronto_df_final = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df_final['PostalCode'] = toronto_df["Postcode"]
toronto_df_final['Borough'] = toronto_df["Borough"]
toronto_df_final['Neighborhood'] = ""

for row in toronto_df_final.index:
    for rows in toronto_df.index:
        if toronto_df_final.iloc[row, 0] == toronto_df.iloc[rows, 0]:
            toronto_df_final.iloc[row, 2] = toronto_df.iloc[rows, 2] + ', ' + toronto_df_final.iloc[row, 2]

toronto_df_final.drop_duplicates(subset=['PostalCode'], inplace = True)
toronto_df_final.reset_index(drop=True, inplace=True)
toronto_df_final

,PostalCode,Borough,Neighborhood
0,M3A,North York,"Parkwoods,"
1,M4A,North York,"Victoria Village,"
2,M5A,Downtown Toronto,"Harbourfront,"
3,M6A,North York,"Lawrence Manor, Lawrence Heights,"
4,M7A,Downtown Toronto,"Queen's Park,"
...,...,...,...
98,M8X,Etobicoke,"Old Mill North, Montgomery Road, The Kingsway,"
99,M4Y,Downtown Toronto,"Church and Wellesley,"
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 East...
101,M8Y,Etobicoke,"Sunnylea, Royal York South East, The Queensway..."


### Obtain dataframe shape

In [46]:
toronto_df_final.shape

(103, 3)

# Part 2

### Obtaining the coordinates of each postal code

In [54]:
lat_long = pd.read_csv('Geospatial_Coordinates.csv')
lat_long.sort_values(by='Postal Code', inplace=True)

toronto_df_final.sort_values(by='PostalCode', inplace=True)
toronto_df_final["Latitude"] = lat_long['Latitude']
toronto_df_final["Longitude"] = lat_long['Longitude']

toronto_df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge,",43.727929,-79.262029
12,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek,",43.794200,-79.262029
18,M1E,Scarborough,"West Hill, Morningside, Guildwood,",43.778517,-79.346556
22,M1G,Scarborough,"Woburn,",43.770120,-79.408493
26,M1H,Scarborough,"Cedarbrae,",43.745906,-79.352188
...,...,...,...,...,...
64,M9N,York,"Weston,",43.696948,-79.411307
70,M9P,Etobicoke,"Westmount,",43.648429,-79.382280
77,M9R,Etobicoke,"St. Phillips, Richview Gardens, Martin Grove G...",43.647927,-79.419750
89,M9V,Etobicoke,"Thistletown, South Steeles, Silverstone, Mount...",43.602414,-79.543484


# Part 3

### Exploring and clustering the data